In [26]:
import pandas as pd
import re
from surprise import accuracy
from surprise import KNNBasic
from surprise import Reader
from surprise import Dataset
import pickle
from collections import defaultdict

In [27]:
column_names1 = ['userId','movieId','rating','timestamp'] 
ratings_dataset = pd.read_csv("C:/Users/DELL/Documents/second semester in 3'rd year/NLP/Movie recommender/ratings.csv",
                            sep=';',header=None,names=column_names1) 
ratings_dataset = ratings_dataset.tail(-1)
ratings_dataset = ratings_dataset.drop('timestamp' , axis = 1)
ratings_dataset=ratings_dataset[0:500000]
ratings_dataset

C:\Users\DELL\AppData\Local\Temp\ipykernel_2268\3485631075.py:2: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_dataset = pd.read_csv("C:/Users/DELL/Documents/second semester in 3'rd year/NLP/Movie recommender/ratings.csv",


,userId,movieId,rating
1,1,1193,5
2,1,661,3
3,1,914,3
4,1,3408,4
5,1,2355,5
...,...,...,...
499996,3069,1584,1
499997,3069,1586,4
499998,3069,1590,4
499999,3069,1591,3


In [28]:
user_movie = pd.pivot_table(ratings_dataset, index = ['userId'], values='movieId', aggfunc='count')
user_movie[user_movie.movieId>0]
user_id = user_movie[user_movie.movieId>0].index


movie_rated = pd.pivot_table(ratings_dataset, index=['movieId'], values='userId', aggfunc='count')
movie_rated[movie_rated.userId>0]
movie_id = movie_rated[movie_rated.userId>0].index


filter_data = ratings_dataset[(ratings_dataset['userId'].isin(user_id)) &(ratings_dataset['movieId'].isin(movie_id))]

filter_data


,userId,movieId,rating
1,1,1193,5
2,1,661,3
3,1,914,3
4,1,3408,4
5,1,2355,5
...,...,...,...
499996,3069,1584,1
499997,3069,1586,4
499998,3069,1590,4
499999,3069,1591,3


In [29]:
column_names1 = ['userId','movieId','rating','timestamp'] 
ratings_dataset = pd.read_csv("C:/Users/DELL/Documents/second semester in 3'rd year/NLP/Movie recommender/ratings.csv",
                            sep=';',header=None,names=column_names1) 
ratings_dataset = ratings_dataset.tail(-1)
 

ratings_dataset = ratings_dataset.drop('timestamp' , axis = 1)
ratings_dataset['userId'] = ratings_dataset['userId'].astype(int)
ratings_dataset['rating'] = ratings_dataset['rating'].astype(int)
ratings_dataset['movieId'] = ratings_dataset['movieId'].astype(int)
print(ratings_dataset.dtypes)
 
matrix = ratings_dataset.pivot(index='userId', columns='movieId', values='rating')
 
matrix = matrix.fillna(0)

matrix

C:\Users\DELL\AppData\Local\Temp\ipykernel_2268\949598542.py:2: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_dataset = pd.read_csv("C:/Users/DELL/Documents/second semester in 3'rd year/NLP/Movie recommender/ratings.csv",


userId     int32
movieId    int32
rating     int32
dtype: object


movieId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(filter_data[['userId','movieId','rating']], reader)

kk = 3
sim_options = {'name': 'cosine', 'user_based': False}
algo = KNNBasic(k = kk, sim_options = sim_options, verbose = True)

#model = NearestNeighbors(metric='cosine', algorithm='brute')
train_set = data.build_full_trainset()
test_set = train_set.build_anti_testset()

algo.fit(train_set)

with open('KNNTrainModel2.pkl', 'wb') as f:
    pickle.dump(algo , f)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [31]:
with open('KNNTrainModel2.pkl', 'rb') as f:
    algo = pickle.load(f)

predictions = algo.test(test_set)

accuracy.rmse(predictions, verbose = True)
#model.fit(train_set)

#pre = model.test(test_set)

RMSE: 0.5549


0.5549057315474226

In [32]:
def get_top_n(predictions, n):
    
    top_movies = defaultdict(list)
    for user, movie, actual_rate, estimate_rate , _ in predictions:
        top_movies[user].append((movie, estimate_rate))
        
    
    for user, user_ratings in top_movies.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_movies[user] = user_ratings[:n]
    
    return top_movies

In [33]:
top_movies = get_top_n(predictions, 10)

In [34]:
with open("C:/Users/DELL/Downloads/movies.csv",  encoding="utf8", errors='ignore') as f:
    contents = f.read() 
lines = contents.split("\n")
 
pattern = re.compile(r'^(\d+);(.*?);(.*)$', re.MULTILINE)
 
genersList = ['Action', 'Adventure', 'Animation', "Children's",
          'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
          'Sci-Fi', 'Thriller', 'War', 'Western']
 
data =  pd.DataFrame()
movie = []
title1 = []
genres1 = []
pattern = re.compile(r'^(\d+);(.*?);(.*);$', re.MULTILINE)
for line in lines:
    for match in pattern.finditer(line):
        movieId = match.group(1)
        title = match.group(2)
       
        if any(item in match.group(3) for item in genersList):
            genres = match.group(3)
            genres1.append(genres)
        else:
            title = title + ','  + match.group(3)
            genres1.append('')
           
        title1.append(title)
        movie.append(movieId)
        
data['movieId'] = movie
data['title'] = title1
data['genres'] = genres1    
    
data = data.drop('genres' , axis = 1)

data

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
3877,3948,Meet the Parents (2000)
3878,3949,Requiem for a Dream (2000)
3879,3950,Tigerland (2000)
3880,3951,Two Family House (2000)


In [36]:
user_id=input("Enter User ID : ")
userID = [str(user_id)]

for user, user_ratings in top_movies.items():
    if user in userID:
        for (movieID, rating) in user_ratings:
            x=data.loc[data['movieId'] == str(movieID)]
            f=list(x['title'])
            movie=f[0]
            print('Movie:', movieID, '-', str(movie))

Enter User ID : 1
Movie: 1527 - Fifth Element, The (1997)
Movie: 3552 - Caddyshack (1980)
Movie: 896 - Wild Reeds (1994)
Movie: 562 - Welcome to the Dollhouse (1995)
Movie: 858 - Godfather, The (1972)
Movie: 2807 - Universal Soldier, The Return (1999)
Movie: 659 - Purple Noon (1960)
Movie: 3521 - Mystery Train (1989)
Movie: 1381 - Grease 2 (1982)
Movie: 3469 - Inherit the Wind (1960)
